In [ ]:
import cv2
import math
import os
import pafy
import random

import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


from collections import deque
from moviepy.editor import *
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical

In [ ]:
## Definições Globais

seed_constant = 23
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

# Diretórios raiz dos datasets
dataset_dir = '../../dataset/'

In [ ]:
## Step 1 & 2 [Download e Visualizar o Dataset]
# Descarregar e descomprimir o dataset para a pasta "datasets" 

# Obter nomes das classes através dos nomes das pastas
all_classes_names = os.listdir(dataset_dir + 'UCF50')

# Criar figura Matplotlib
plt.figure(figsize = (30, 30))

# Gerar conjunto aleatório de imagens sempre que a célula correr
random_range = random.sample(range(len(all_classes_names)), 20)

# Iterar todos os elementos do conjunto aleatório
for counter, random_index in enumerate(random_range, 1):

    # Obter nome da classe
    selected_class_Name = all_classes_names[random_index]
    
    # Obter lista de ficheiros video presentes na pasta da classe
    video_files_names_list = os.listdir(dataset_dir + f'UCF50/{selected_class_Name}')

    # Selecionar video aleatório
    selected_video_file_name = random.choice(video_files_names_list)
    
    # Ler ficheiro de vídeo usando o VideoCapture
    video_reader = cv2.VideoCapture(dataset_dir + f'UCF50/{selected_class_Name}/{selected_video_file_name}')

    # Ler primeira Frame do ficheiro de vídeo
    _, bgr_frame = video_reader.read()

    # Fechar o objecto VideoCapture para libertar recursos
    video_reader.release()

    # Converter a Frame do vídeo de BGR para RGB
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

    # Adicionar o nome da classe ao topo da Frame de vídeo
    cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Associar a Frame a uma posição específica do subplot
    plt.subplot(5, 4, counter)
    plt.imshow(rgb_frame)
    plt.axis('off')

In [ ]:
## Step 3 [Ler e Pré-processar Dataset]

image_height, image_width = 64, 64
max_images_per_class = 8000
dataset_name = "UCF50"
classes_list = ["WalkingWithDog", "TaiChi", "Swing", "HorseRace"]
model_output_size = len(classes_list)

def frames_extraction(video_path):
    
    frames_list = []
    
    # Ler ficheiro de vídeo usando o VideoCapture
    video_reader = cv2.VideoCapture(video_path)
    
    # Iterar as frames do vídeo
    while True:
        # Ler frame do ficheiro de vídeo
        success, frame = video_reader.read()
        
        # Se não ler frame de vídeo com sucesso, quebra o loop
        if not success:
            break
            
        # Redimensionar frame para as dimensões especificadas
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        # Normalizar a frame, dividindo por 255, para que cada valor de pixel seja entre 0 e 1
        normalized_frame = resized_frame / 255
        
        # Adicionar a frame normalizada à lista de frames
        frames_list.append(normalized_frame)
        
    # Fechar o objecto VideoCapture para libertar recursos
    video_reader.release()

    return frames_list

def create_dataset():
    
    temp_features = []
    features = []
    labels = []
    
    # Iterar todas as classes
    for class_index, class_name in enumerate(classes_list):
        print(f'Extracting Data of Class: {class_name}')

        # Obter lista de ficheiros video presentes na pasta da classe
        files_list = os.listdir(os.path.join(dataset_dir + dataset_name, class_name))

        # Iterar todos os ficheiros presentes na lista
        for file_name in files_list:

            # Preparar o caminho do ficheiro
            video_file_path = os.path.join(dataset_dir + dataset_name, class_name, file_name)

            # Extrair as frames do ficheiro vídeo
            frames = frames_extraction(video_file_path)

            # Adicionar as frames a uma lista temporária
            temp_features.extend(frames)
            
        # Adicionar frames aleatórias à lista das features
        features.extend(random.sample(temp_features, max_images_per_class))
        
        # Adicionar números fixos das labels à lista das labels
        labels.extend([class_index] * max_images_per_class)
        
        # Esvaziar lista temporária para que possa ser reutilizada
        temp_features.clear()
        
    # Converter listas de features e labels em arrays numpy
    features = np.asarray(features)
    labels = np.array(labels) 
    
    return features, labels

In [ ]:
# Criar dataset
features, labels = create_dataset()

# Converter labels para formato one_hot_encoded
one_hot_encoded_labels = to_categorical(labels)

In [ ]:
## Step 4 [Dividir dataset em Sets de treino e test]

# Split de Treino e Teste de 80% e 20%
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.2, shuffle = True, random_state = seed_constant)

In [ ]:
## Step 5 [Construir Modelo]

def create_model():
    # Vai-se utilizar um modelo sequencial
    model = Sequential()
    
    # Definir a arquitetura do modelo
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = (image_height, image_width, 3)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(model_output_size, activation = 'softmax'))
    
    # Imprimir sumário do modelo
    model.summary()
    return model

# Criar modelo
model = create_model()

print("Model Created Successfully!")

In [ ]:
# Verificar estrutura do modelo
plot_model(model, to_file = 'model_structure_plot.png', show_shapes = True, show_layer_names = True)

In [ ]:
## Step 6 [Compilar e Treinar o Modelo]

# Adicionar Callback para Early Stopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

# Adicionar perda, optimizador e valores de métricas ao modelo
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Iniciar Treino
epochs = 50
batch_size = 4

model_training_history = model.fit(x = features_train, y = labels_train, epochs = epochs, batch_size = batch_size , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

In [ ]:
# Avaliar Modelo
model_evaluation_history = model.evaluate(features_test, labels_test)

In [ ]:
# Criar nome útil para o modelo no caso de haver múltiplos modelos
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history
model_name = f'Model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

# Gravar Modelo
model.save(model_name)

In [ ]:
## Step 7 [Plot das curvas de perda e precisão]

def plot_metric(metric_name_1, metric_name_2, plot_name):
    # Obter valores das métricas usando nomes como identificadores
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]
    
    # Criar objecto range que servirá de tempo
    epochs = range(len(metric_value_1))
    
    # Plotar o gráfico
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Adicionar título ao plot
    plt.title(str(plot_name))
    
    # Adicionar legenda ao plot
    plt.legend()

In [ ]:
plot_metric('loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
plot_metric('accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy')

In [ ]:
## Step 8 [Fazer Previsões usando o modelo]

def download_youtube_videos(youtube_video_url, output_directory):
    # Creating a Video object which includes useful information regarding the youtube video.
    video = pafy.new(youtube_video_url)

    # Getting the best available quality object for the youtube video.
    video_best = video.getbest()

    # Constructing the Output File Path
    output_file_path = f'{output_directory}/{video.title}.mp4'
    
    # Downloading the youtube video at the best available quality.
    video_best.download(filepath = output_file_path, quiet = True)

    # Returning Video Title
    return video.title

In [ ]:
def predict_on_live_video(video_file_path, output_file_path, window_size):
    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    predicted_labels_probabilities_deque = deque(maxlen = window_size)

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)

    # Getting the width and height of the video
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True:
        # Reading The Frame
        status, frame = video_reader.read()
        
        if not status:
            break

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]

        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)

        # Assuring that the Deque is completely filled before starting the averaging process
        if len(predicted_labels_probabilities_deque) == window_size:
        
            # Converting Predicted Labels Probabilities Deque into Numpy array
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)

            # Calculating Average of Predicted Labels Probabilities Column Wise
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)

            # Converting the predicted probabilities into labels by returning the index of the maximum value.
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)

            # Accessing The Class Name using predicted label.
            predicted_class_name = classes_list[predicted_label]

            # Overlaying Class Name Text Ontop of the Frame
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Writing The Frame
        video_writer.write(frame)

        # cv2.imshow('Predicted Frames', frame)
        # key_pressed = cv2.waitKey(10)
        # if key_pressed == ord('q'):
        #     break
    # cv2.destroyAllWindows()

    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them.
    video_reader.release()
    video_writer.release()

In [ ]:
# Creating The Output directories if it does not exist
output_directory = 'Youtube_Videos'
os.makedirs(output_directory, exist_ok = True)

# Downloading a YouTube Video
video_title = download_youtube_videos('https://www.youtube.com/watch?v=8u0qjmHIOcE', output_directory)
 
# Getting the YouTube Video's path you just downloaded
input_video_file_path = f'{output_directory}/{video_title}.mp4'

In [ ]:
# Setting sthe Window Size which will be used by the Rolling Average Proces
window_size = 1

# Constructing The Output YouTube Video Path
output_video_file_path = f'{output_directory}/{video_title} -Output-WSize {window_size}.mp4'

# Calling the predict_on_live_video method to start the Prediction.
predict_on_live_video(input_video_file_path, output_video_file_path, window_size)

# Play Video File in the Notebook
VideoFileClip(output_video_file_path).ipython_display(width = 700)


In [ ]:
# Setting the Window Size which will be used by the Rolling Average Process
window_size = 25

# Constructing The Output YouTube Video Path
output_video_file_path = f'{output_directory}/{video_title} -Output-WSize {window_size}.mp4'

# Calling the predict_on_live_video method to start the Prediction and Rolling Average Process
predict_on_live_video(input_video_file_path, output_video_file_path, window_size)

# Play Video File in the Notebook
VideoFileClip(output_video_file_path).ipython_display(width = 700)

In [ ]:
## Step 9 [Usando método de CNN frame singular]

def make_average_predictions(video_file_path, predictions_frames_count):
    # Initializing the Numpy array which will store Prediction Probabilities
    predicted_labels_probabilities_np = np.zeros((predictions_frames_count, model_output_size), dtype = np.float)

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)

    # Getting The Total Frames present in the video
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculating The Number of Frames to skip Before reading a frame
    skip_frames_window = video_frames_count // predictions_frames_count

    for frame_counter in range(predictions_frames_count):

        # Setting Frame Position
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading The Frame
        _ , frame = video_reader.read()

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]

        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_np[frame_counter] = predicted_labels_probabilities

    # Calculating Average of Predicted Labels Probabilities Column Wise
    predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)

    # Sorting the Averaged Predicted Labels Probabilities
    predicted_labels_probabilities_averaged_sorted_indexes = np.argsort(predicted_labels_probabilities_averaged)[::-1]

    # Iterating Over All Averaged Predicted Label Probabilities
    for predicted_label in predicted_labels_probabilities_averaged_sorted_indexes:

        # Accessing The Class Name using predicted label.
        predicted_class_name = classes_list[predicted_label]

        # Accessing The Averaged Probability using predicted label.
        predicted_probability = predicted_labels_probabilities_averaged[predicted_label]

        print(f"CLASS NAME: {predicted_class_name}   AVERAGED PROBABILITY: {(predicted_probability*100):.2}")

    # Closing the VideoCapture Object and releasing all resources held by it.
    video_reader.release()

In [ ]:
# Downloading The YouTube Video
video_title = download_youtube_videos('https://www.youtube.com/watch?v=ceRjxW4MpOY', output_directory)

# Constructing The Input YouTube Video Path
input_video_file_path = f'{output_directory}/{video_title}.mp4'

# Calling The Make Average Method To Start The Process
make_average_predictions(input_video_file_path, 50)

# Play Video File in the Notebook
VideoFileClip(input_video_file_path).ipython_display(width = 700)

In [ ]:
# Downloading The YouTube Video
video_title = download_youtube_videos('https://www.youtube.com/watch?v=ayI-e3cJM-0', output_directory)

# Constructing The Input YouTube Video Path
input_video_file_path = f'{output_directory}/{video_title}.mp4'

# Calling The Make Average Method To Start The Process
make_average_predictions(input_video_file_path, 50)

# Play Video File in the Notebook
VideoFileClip(input_video_file_path).ipython_display(width = 700)

In [ ]:
# Downloading The YouTube Video
video_title = download_youtube_videos('https://www.youtube.com/watch?v=XqqpZS0c1K0', output_directory)

# Constructing The Input YouTube Video Path
input_video_file_path = f'{output_directory}/{video_title}.mp4'

# Calling The Make Average Method To Start The Process
make_average_predictions(input_video_file_path, 50)

# Play Video File in the Notebook
VideoFileClip(input_video_file_path).ipython_display(width = 700)

In [ ]:
# Downloading The YouTube Video
video_title = download_youtube_videos('https://www.youtube.com/watch?v=WHBu6iePxKc', output_directory)

# Constructing The Input YouTube Video Path
input_video_file_path = f'{output_directory}/{video_title}.mp4'

# Calling The Make Average Method To Start The Process
make_average_predictions(input_video_file_path, 50)

# Play Video File in the Notebook
VideoFileClip(input_video_file_path).ipython_display(width = 700)